In [110]:
'''Predicting Mortality across Germany wih different AI Methods
'''

import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import impute
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import dbf
from pygam import GAM, s, f, LinearGAM
import xgboost
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Load Data

In [111]:
# Set the working directory
os.chdir("N:/WG_ENRI/20_projects/Noise2NAKO/04_data/grid_prediction/3_output")

In [112]:
# Read SES training and test data
data_training = pd.read_csv('SES_training.csv', sep=',')
data_test = pd.read_csv('SES_test.csv', sep=',')

In [125]:
# distinguish between response and input variables and make training and test data 
# remove unnecessary features 'id', 'x_sw', 'y_sw','x_mp', 'y_mp'

y = data_training['ks_Mortality_17']
X = data_training.drop(['ks_Mortality_17', 'id', 'x_sw', 'y_sw','x_mp', 'y_mp'], axis = 1)

pred_y = data_test['ks_Mortality_17']
pred_X = data_test.drop(['ks_Mortality_17', 'id', 'x_sw', 'y_sw','x_mp', 'y_mp'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

In [130]:
result= pd.DataFrame(columns=['Method', 'MSE_train', 'MAE_train', 'R_2_train', 'MSE_test', 'MAE_test', 'R_2_test', 'MSE_pred', 'MAE_pred', 'R_2_pred'])

# Prediction

In [127]:
def Prediction(method, X_train, X_test, y_train, y_test, pred_X, pred_y):
    
    '''Prediction Function'''
    
    if method == 'LR':
        model = linear_model.LinearRegression().fit(X_train, y_train)
    elif method == 'GAM':
        model = GAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9)).fit(X_train, y_train)
    elif method == 'RF':
        model = RandomForestRegressor(n_estimators= 10, max_depth= 10, random_state= 0, max_features='sqrt').fit(X_train, y_train)
    else:
        model = xgboost.XGBRegressor().fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    y_pred = model.predict(pred_X)

    result = {'Method': method, 'MSE_train': mean_squared_error(y_train, y_pred_train),
                            'MAE_train': mean_absolute_error(y_train, y_pred_train), 'R_2_train': r2_score(y_train, y_pred_train),
                           'MSE_test': mean_squared_error(y_test, y_pred_test), 'MAE_test': mean_absolute_error(y_test, y_pred_test),
                           'R_2_test': r2_score(y_test, y_pred_test), 'MSE_pred': mean_squared_error(pred_y, y_pred), 'MAE_pred': mean_absolute_error(pred_y, y_pred),
                           'R_2_pred': r2_score(pred_y, y_pred)}
    return result

### Prediction with different methods

In [131]:
# Linear Regression
result = result.append(Prediction(method='LR', X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, pred_X=pred_X, pred_y=pred_y), ignore_index=True)

# GAM
result = result.append(Prediction(method='GAM', X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, pred_X=pred_X, pred_y=pred_y), ignore_index=True)

# Random Forest
result = result.append(Prediction(method='RF', X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, pred_X=pred_X, pred_y=pred_y), ignore_index=True)

# XGBoost
result = result.append(Prediction(method='XGBoost', X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, pred_X=pred_X, pred_y=pred_y), ignore_index=True)

result

,Method,MSE_train,MAE_train,R_2_train,MSE_test,MAE_test,R_2_test,MSE_pred,MAE_pred,R_2_pred
0,LR,0.384853,0.426614,0.893768,0.361315,0.409386,0.900386,1.567778,0.955292,0.536930
1,GAM,0.091077,0.183962,0.974860,0.125594,0.208670,0.965374,1.678240,0.996681,0.504303
2,RF,0.030275,0.077560,0.991643,0.123404,0.161132,0.965978,1.330406,0.880585,0.607041
3,XGBoost,0.000011,0.002114,0.999997,0.098933,0.126608,0.972724,1.323051,0.872430,0.609214


In [132]:
result.to_csv('PredictionTask_results.csv', index=False, sep=',')